In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path_dict = {
    "read_air_reserve" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/air_reserve.csv",
    "read_air_store_info" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/air_store_info.csv",
    "read_air_visit_data" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/air_visit_data.csv",
    "read_date_info" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/date_info.csv",
    "read_hpg_reserve" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/hpg_reserve.csv",
    "read_hpg_store_info" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/hpg_store_info.csv",
    "read_sample_submission": "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/sample_submission.csv",
    "read_store_id_relation": "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/store_id_relation.csv",
    "read_weather_df" : "weather_df.csv",
    "read_air_store_info_kmeans" : "air_store_info_kmeans.csv"
}

# 0.0 Load the data

In [3]:
air_visit_data_df = pd.read_csv(path_dict["read_air_visit_data"])
date_info_df = pd.read_csv(path_dict["read_date_info"])
sample_submission_df = pd.read_csv(path_dict["read_sample_submission"])


In [4]:
# extract the id from the id_data concat

# 0.1 modify sample submission file

In [5]:
sample_submission_df["air_store_id"] = sample_submission_df["id"].apply(lambda x: x[:20])

In [6]:
sample_submission_df["visit_date"] = sample_submission_df["id"].apply(lambda x: x[21:])

# 1.0 extract date related info

In [7]:
# merge with the data containing info on holiday, dya of the week
def extract_data_features(air_visit_data_df, date_info_df):
    
    air_visit_data_df = pd.merge(air_visit_data_df, date_info_df, left_on = "visit_date", right_on = "calendar_date", how = "left")
    air_visit_data_df.drop("calendar_date", axis = 1, inplace=True)

    # transform visit_date to datetime
    air_visit_data_df["visit_date"] = pd.to_datetime(air_visit_data_df["visit_date"])

    # create variable year
    air_visit_data_df["year"] = air_visit_data_df["visit_date"].apply(lambda x: x.year)
    # create variable month
    air_visit_data_df["month"] = air_visit_data_df["visit_date"].apply(lambda x: x.month)

    # apply one hot encoding
    air_visit_data_df["month"] = air_visit_data_df["month"].astype(str)
    air_visit_data_df["year"] = air_visit_data_df["year"].astype(str)

    ohe_day_of_week = pd.get_dummies(air_visit_data_df[["day_of_week"]])
    ohe_year = pd.get_dummies(air_visit_data_df[["year"]])
    ohe_month = pd.get_dummies(air_visit_data_df[["month"]])
    '''
    print(ohe_day_of_week.head())
 
    print(ohe_year.head())
 
    print(ohe_month.head())
    '''
    air_visit_data_df = pd.concat([air_visit_data_df, ohe_day_of_week], axis = 1)
    air_visit_data_df = pd.concat([air_visit_data_df, ohe_year], axis = 1)
    air_visit_data_df = pd.concat([air_visit_data_df, ohe_month], axis = 1)
    
    return air_visit_data_df

In [8]:
air_visit_data_df = extract_data_features(air_visit_data_df, date_info_df)
sample_submission_df = extract_data_features(sample_submission_df, date_info_df)

In [9]:
sample_submission_df.head()

,id,visitors,air_store_id,visit_date,day_of_week,holiday_flg,year,month,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,year_2017,month_4,month_5
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Sunday,0,2017,4,0,0,0,1,0,0,0,1,1,0
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Monday,0,2017,4,0,1,0,0,0,0,0,1,1,0
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,2017,4,0,0,0,0,0,1,0,1,1,0
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,2017,4,0,0,0,0,0,0,1,1,1,0
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Thursday,0,2017,4,0,0,0,0,1,0,0,1,1,0


In [10]:
air_visit_data_df.head()

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,year,month,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,...,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,2016,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,2016,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,2016,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,2016,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,2016,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
list(air_visit_data_df.columns)

['air_store_id',
 'visit_date',
 'visitors',
 'day_of_week',
 'holiday_flg',
 'year',
 'month',
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday',
 'year_2016',
 'year_2017',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9']

# 2.0 dumb model

In [27]:
from sklearn import ensemble, neighbors, linear_model, metrics, preprocessing

In [44]:
col = [ 
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday',
 'year_2016',
 'year_2017',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9']

In [40]:
# add columns that are missing
for column in [ 
 'year_2016',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3']:
    sample_submission_df[column] = 0

In [34]:
# define models
etc = ensemble.ExtraTreesRegressor(n_estimators=225, max_depth=5, n_jobs=-1, random_state=3)
knn = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)


In [35]:
# fit models
etc.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)
knn.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
          weights='uniform')

In [45]:
etc.predict(sample_submission_df[col])

array([ 23.66518505,  17.83419804,  17.83419804, ...,  17.83419804,
        17.83419804,  17.83419804])

In [46]:
sample_submission_df['visitors'] = (etc.predict(sample_submission_df[col]) / 2) +(knn.predict(sample_submission_df[col]) / 2)

In [49]:
sample_submission_df[["id", "visitors"]].to_csv("dumb_submission.csv",index = False)

# 1.1 extract information regarding the number of visitors of each particular restaurant on the day of the week

In [51]:
def extract_mean_rest_day_features(air_visit_data_df):
    air_visit_by_rest_day_df = air_visit_data_df.groupby(["air_store_id", "day_of_week","holiday_flg"])["visitors"].mean().reset_index()
    air_visit_by_rest_day_df.rename(columns = {"visitors" : "visitors_resto_day"}, inplace = True)
    air_visit_by_rest_day_df.head()
    air_visit_data_df = pd.merge(air_visit_data_df, air_visit_by_rest_day_df, on = ["air_store_id","day_of_week","holiday_flg"], how = "left")
    air_visit_data_df.head()
    air_visit_data_df["visitors_resto_day"] = air_visit_data_df["visitors_resto_day"].apply(lambda x : math.ceil(x))
    return air_visit_data_df

In [58]:
air_visit_data_df = extract_mean_rest_day_features(air_visit_data_df)

In [52]:
sample_submission_df = extract_mean_rest_day_features(sample_submission_df)

# 2.1 dumb model 2

In [53]:
from sklearn import ensemble, neighbors, linear_model, metrics, preprocessing

In [54]:
# define models
etc = ensemble.ExtraTreesRegressor(n_estimators=225, max_depth=5, n_jobs=-1, random_state=3)
knn = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)


In [55]:
col = [ 
 "visitors_resto_day",
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday',
 'year_2016',
 'year_2017',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9']

In [56]:
# add columns that are missing
for column in [ 
 'year_2016',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3']:
    sample_submission_df[column] = 0

In [59]:
# fit models
etc.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)
knn.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
          weights='uniform')

In [60]:
sample_submission_df['visitors'] = (etc.predict(sample_submission_df[col]) / 2) +(knn.predict(sample_submission_df[col]) / 2)

In [61]:
sample_submission_df[["id", "visitors"]].to_csv("dumb_submission_2.csv",index = False)

# 1.3 add features related to the type and neighbourhood of the restarant

In [9]:
air_store_info_df = pd.read_csv(path_dict["read_air_store_info"])
air_store_info_df.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [10]:
pd.get_dummies(air_store_info_df[["air_area_name"]])

,air_area_name_Fukuoka-ken Fukuoka-shi Daimyō,air_area_name_Fukuoka-ken Fukuoka-shi Hakata Ekimae,air_area_name_Fukuoka-ken Fukuoka-shi Imaizumi,air_area_name_Fukuoka-ken Fukuoka-shi Momochi,air_area_name_Fukuoka-ken Fukuoka-shi Shiobaru,air_area_name_Fukuoka-ken Fukuoka-shi Takatori,air_area_name_Fukuoka-ken Fukuoka-shi Tenjin,air_area_name_Fukuoka-ken Fukuoka-shi Torikai,air_area_name_Fukuoka-ken Itoshima-shi Maebarunishi,air_area_name_Fukuoka-ken Kitakyūshū-shi Konyamachi,...,air_area_name_Ōsaka-fu Sakai-shi Minamikawaramachi,air_area_name_Ōsaka-fu Suita-shi Izumichō,air_area_name_Ōsaka-fu Ōsaka-shi Fuminosato,air_area_name_Ōsaka-fu Ōsaka-shi Kyōmachibori,air_area_name_Ōsaka-fu Ōsaka-shi Kyūtarōmachi,air_area_name_Ōsaka-fu Ōsaka-shi Nakanochō,air_area_name_Ōsaka-fu Ōsaka-shi Nanbasennichimae,air_area_name_Ōsaka-fu Ōsaka-shi Shinmachi,air_area_name_Ōsaka-fu Ōsaka-shi Ōgimachi,air_area_name_Ōsaka-fu Ōsaka-shi Ōhiraki
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# one hot encoding for restaurant type and neighbourhood
def extract_mean_rest_day_features(air_store_info_df):
    ohe_air_genre_name = pd.get_dummies(air_store_info_df[["air_genre_name"]])
    ohe_air_area_name  = pd.get_dummies(air_store_info_df[["air_area_name"]])

    air_store_info_df = pd.concat([air_store_info_df, ohe_air_genre_name], axis = 1)
    air_store_info_df = pd.concat([air_store_info_df, ohe_air_area_name], axis = 1)
    return air_store_info_df

In [12]:
air_store_info_df = extract_mean_rest_day_features(air_store_info_df)

In [13]:
air_visit_data_df = pd.merge(air_store_info_df, air_visit_data_df, on = "air_store_id", how = "right")
sample_submission_df = pd.merge(air_store_info_df, sample_submission_df, on = "air_store_id", how = "right")

# 2.3 dumb model with resto genre and location

In [71]:
col = [ 
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday',
 'year_2016',
 'year_2017',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9']

In [73]:
col.extend(list(pd.get_dummies(air_store_info_df[["air_genre_name"]]).columns))

In [74]:
# fit models
etc.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)
knn.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
          weights='uniform')

In [75]:
sample_submission_df['visitors'] = (etc.predict(sample_submission_df[col]) / 2) +(knn.predict(sample_submission_df[col]) / 2)

In [76]:
sample_submission_df[["id", "visitors"]].to_csv("dumb_submission_3.csv",index = False)

In [80]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV

/home/andraa/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/andraa/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [83]:
clf_rfr = RandomForestRegressor()
# parametres à regler via validation croissée
params_rf = {
 'n_estimators': [100, 200, 500, 1000],            
 'max_features': ['auto', 'sqrt', 'log2'],
 "bootstrap": [True, False], 'verbose' : [1]          
}
    
clf_cv_grid = GridSearchCV( estimator = clf_rfr, param_grid = params_rf, 
                   cv = 5, verbose = 1, n_jobs=-1, refit=True)

clf_cv_grid.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.0min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  3.5min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  2.7min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    7.0s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  4.0min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    6.2s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  3.3min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    3.5s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  1.8min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.9s finished
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 92.3min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 200, 500, 1000], 'max_features': ['auto', 'sqrt', 'log2'], 'bootstrap': [True, False], 'verbose': [1]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [84]:
sample_submission_df["visitors"] = clf_cv_grid.predict(sample_submission_df[col])

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.9s finished


In [85]:
sample_submission_df[["id", "visitors"]].to_csv("dumb_submission_4.csv",index = False)

# Weather Data features

In [14]:
weather_df = pd.read_csv(path_dict["read_weather_df"], index_col =0)

In [22]:
weather_df["date"] = pd.to_datetime(weather_df["date"])

In [23]:
weather_df

,date,icon,moonPhase,temperatureMax,temperatureMin,windSpeed,cluster
0,2016-01-01,clear-day,0.72,52.48,36.63,4.35,3
1,2016-01-01,clear-day,0.72,51.85,34.97,0.55,5
2,2016-01-01,clear-day,0.72,50.73,39.69,2.26,8
3,2016-01-01,clear-day,0.72,50.96,37.21,2.97,0
4,2016-01-01,clear-day,0.72,50.52,36.33,7.74,6
5,2016-01-01,clear-day,0.72,53.39,38.66,5.72,1
6,2016-01-01,clear-day,0.72,43.18,34.48,5.23,7
7,2016-01-01,clear-day,0.72,41.96,30.46,4.31,4
8,2016-01-01,clear-day,0.72,31.18,27.58,5.08,2
9,2016-01-01,clear-day,0.72,27.80,22.44,3.80,9


In [16]:
air_store_info_df = pd.read_csv("air_store_info_kmeans.csv")

In [17]:
air_store_info_df.head()

,Unnamed: 0,air_store_id,air_genre_name,air_area_name,latitude,longitude,cluster
0,0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,8
1,1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,8
2,2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,8
3,3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,8
4,4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1


In [18]:
air_visit_data_df 
sample_submission_df

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_genre_name_Asian,air_genre_name_Bar/Cocktail,air_genre_name_Cafe/Sweets,air_genre_name_Creative cuisine,air_genre_name_Dining bar,...,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,year_2017,month_4,month_5
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
1,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
2,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,0
3,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
4,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
5,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
6,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
7,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
8,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
9,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [19]:
# first we do the merge with the air_stroe_info_df that contains the kmeans info
air_visit_data_df = pd.merge(air_visit_data_df,air_store_info_df[["air_store_id","cluster"]], on = "air_store_id")
sample_submission_df = pd.merge(sample_submission_df, air_store_info_df[["air_store_id","cluster"]], on = "air_store_id")


In [18]:
sample_submission_df

,id,visitors,air_store_id,visit_date,day_of_week,holiday_flg,year,month,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,year_2017,month_4,month_5,cluster
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Sunday,0,2017,4,0,0,0,1,0,0,0,1,1,0,1
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Monday,0,2017,4,0,1,0,0,0,0,0,1,1,0,1
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,2017,4,0,0,0,0,0,1,0,1,1,0,1
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,2017,4,0,0,0,0,0,0,1,1,1,0,1
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Thursday,0,2017,4,0,0,0,0,1,0,0,1,1,0,1
5,air_00a91d42b08b08d9_2017-04-28,0,air_00a91d42b08b08d9,2017-04-28,Friday,0,2017,4,1,0,0,0,0,0,0,1,1,0,1
6,air_00a91d42b08b08d9_2017-04-29,0,air_00a91d42b08b08d9,2017-04-29,Saturday,1,2017,4,0,0,1,0,0,0,0,1,1,0,1
7,air_00a91d42b08b08d9_2017-04-30,0,air_00a91d42b08b08d9,2017-04-30,Sunday,0,2017,4,0,0,0,1,0,0,0,1,1,0,1
8,air_00a91d42b08b08d9_2017-05-01,0,air_00a91d42b08b08d9,2017-05-01,Monday,0,2017,5,0,1,0,0,0,0,0,1,0,1,1
9,air_00a91d42b08b08d9_2017-05-02,0,air_00a91d42b08b08d9,2017-05-02,Tuesday,0,2017,5,0,0,0,0,0,1,0,1,0,1,1


In [24]:
# secondly we merge with the weather data by cluster and day
air_visit_data_df = pd.merge(air_visit_data_df, weather_df, left_on = ["visit_date","cluster"], right_on = ["date","cluster"])
sample_submission_df = pd.merge(sample_submission_df, weather_df,left_on = ["visit_date","cluster"], right_on = ["date","cluster"])


In [25]:
air_visit_data_df

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_genre_name_Asian,air_genre_name_Bar/Cocktail,air_genre_name_Cafe/Sweets,air_genre_name_Creative cuisine,air_genre_name_Dining bar,...,month_7,month_8,month_9,cluster,date,icon,moonPhase,temperatureMax,temperatureMin,windSpeed
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
3,air_3440e0ea1b70a99b,Italian/French,Ōsaka-fu Ōsaka-shi Ōgimachi,34.705362,135.510025,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
4,air_722297e7f26db91d,Italian/French,Ōsaka-fu Ōsaka-shi Ōgimachi,34.705362,135.510025,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
5,air_d00a15343325e5f7,Italian/French,Ōsaka-fu Suita-shi Izumichō,34.759453,135.516860,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
6,air_228f10bec0bda9c8,Italian/French,Ōsaka-fu Suita-shi Izumichō,34.759453,135.516860,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
7,air_d1f20424f76cc78e,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
8,air_32f5d7cd696e3c4a,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28
9,air_40f6193ea3ed1b91,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,0,0,0,0,0,...,1,0,0,8,2016-07-01,clear-day,0.88,87.31,72.81,3.28


In [27]:
# one hot encoding on icon 
def ohe_icon(air_store_info_df):
    ohe_air_icon  = pd.get_dummies(air_store_info_df[["icon"]])
    air_store_info_df = pd.concat([air_store_info_df, ohe_air_icon], axis = 1)
    return air_store_info_df
air_visit_data_df = ohe_icon(air_visit_data_df)
sample_submission_df = ohe_icon(sample_submission_df)

# 2.4 dumb model with resto genre and location and weather

In [44]:
col = [ 
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday',
 'year_2016',
 'year_2017',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 "moonPhase",
 "temperatureMax",
 "temperatureMin",
 "windSpeed",
"cluster"]
col.extend(list(pd.get_dummies(air_store_info_df[["air_genre_name"]]).columns))


In [45]:
col

['day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday',
 'year_2016',
 'year_2017',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'moonPhase',
 'temperatureMax',
 'temperatureMin',
 'windSpeed',
 'cluster',
 'air_genre_name_Asian',
 'air_genre_name_Bar/Cocktail',
 'air_genre_name_Cafe/Sweets',
 'air_genre_name_Creative cuisine',
 'air_genre_name_Dining bar',
 'air_genre_name_International cuisine',
 'air_genre_name_Italian/French',
 'air_genre_name_Izakaya',
 'air_genre_name_Japanese food',
 'air_genre_name_Karaoke/Party',
 'air_genre_name_Okonomiyaki/Monja/Teppanyaki',
 'air_genre_name_Other',
 'air_genre_name_Western food',
 'air_genre_name_Yakiniku/Korean food']

In [46]:
col.extend(list(pd.get_dummies(air_visit_data_df[["icon"]]).columns))

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
clf_rfr = RandomForestRegressor()
# parametres à regler via validation croissée
params_rf = {
 'n_estimators': [100, 200, 500, 1000],            
 'max_features': ['auto', 'sqrt', 'log2'],
 "bootstrap": [True, False], 'verbose' : [1]          
}
    
clf_cv_grid = GridSearchCV( estimator = clf_rfr, param_grid = params_rf, 
                   cv = 5, verbose = 1, n_jobs=-1, refit=True)

clf_cv_grid.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/home/andraa/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/andraa/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.1min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.1min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elap

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.3min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.8min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.7s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  4.5min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    7.0s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  4.7min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  4.8min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.6s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.6s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  4.3min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    7.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.8min finished
[Par

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  3.0min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.9min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.8min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    3.6s finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.2min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.4s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  5.7min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    7.4s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  6.0min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  5.9min finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.5s finished
[Paralle

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 200, 500, 1000], 'max_features': ['auto', 'sqrt', 'log2'], 'bootstrap': [True, False], 'verbose': [1]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [47]:
sample_submission_df["visitors"] = clf_cv_grid.predict(sample_submission_df[col])

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    1.4s finished


In [35]:
unknown_col = ['year_2016', 'month_1', 'month_10', 'month_11', 'month_12', 'month_2', 'month_3', 'month_6', 'month_7', 'month_8', 'month_9', 'icon_cloudy']

In [36]:
for col in unknown_col:
    sample_submission_df[col] =0

In [48]:
sample_submission_df[["id", "visitors"]].to_csv("dumb_submission_4.csv",index = False)

In [39]:
sample_submission_df.shape

(32019, 164)

In [40]:
sample_submission_df.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_genre_name_Asian,air_genre_name_Bar/Cocktail,air_genre_name_Cafe/Sweets,air_genre_name_Creative cuisine,air_genre_name_Dining bar,...,month_10,month_11,month_12,month_2,month_3,month_6,month_7,month_8,month_9,icon_cloudy
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,air_3440e0ea1b70a99b,Italian/French,Ōsaka-fu Ōsaka-shi Ōgimachi,34.705362,135.510025,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
len(col)

11